In [5]:
import spacy
import matplotlib.pyplot as plt
import pandas as pd
from wordcloud import WordCloud
from collections import Counter
from nltk.util import ngrams
import os
from sklearn.feature_extraction.text import CountVectorizer
nlp = spacy.load("en_core_web_sm",disable=["ner", "parser"])

Pre procesamiento

In [8]:
def limpieza(textos):
    resultados = []
    # Convertir todo a minúsculas antes de procesar
    textos = [texto.lower() for texto in textos]
    # Procesar con spacy
    for doc in nlp.pipe(textos, batch_size=1000):
        #Conservar solo la lematizacion de tokens alfabéticos y no stopwords, 
        palabras = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop and len(token.lemma_) >1]
        resultados.append(palabras)
    return resultados

In [13]:
data = pd.read_csv('../data/news_dataset.zip', compression='zip') 
data.head()

,text,label
0,"Donald Trump s White House is in chaos, and th...",0
1,Now that Donald Trump is the presumptive GOP n...,0
2,Mike Pence is a huge homophobe. He supports ex...,0
3,SAN FRANCISCO (Reuters) - California Attorney ...,1
4,Twisted reasoning is all that comes from Pelos...,0


In [14]:
#Buscar nulos y duplicados 
print("Nulos:\n",data.isnull().sum())
print("Duplicados: ",data.duplicated().sum())

#Eliminar duplicados 
data.drop_duplicates(inplace=True) 
print("Duplicados: ",data.duplicated().sum())

Nulos:
 text     0
label    0
dtype: int64
Duplicados:  6251
Duplicados:  0


In [16]:
#Procesar en chunks 
chunk_size = 5000  # Cambia según tu RAM
textos = list(data['text'])
limpiar_textos = []

print("Procesando textos en chunks")
for i in range(0, len(textos), chunk_size):
    chunk = textos[i:i+chunk_size]
    limpiar_chunk = limpieza(chunk)
    limpiar_textos.extend(limpiar_chunk)
#Aplicar limpieza
print(limpiar_textos[:5])

Procesando textos en chunks
[['donald', 'trump', 'white', 'house', 'chaos', 'try', 'cover', 'russia', 'problem', 'mount', 'hour', 'refuse', 'acknowledge', 'problem', 'surround', 'fake', 'news', 'hoax', 'fact', 'bear', 'thing', 'differently', 'crack', 'congressional', 'public', 'grassley', 'iowa', 'head', 'senate', 'judiciary', 'committee', 'feed', 'demand', 'donald', 'trump', 'jr', 'trump', 'campaign', 'manager', 'paul', 'manafort', 'testify', 'committee', 'infamous', 'shady', 'meeting', 'donald', 'trump', 'shady', 'russian', 'lawyer', 'promise', 'dirt', 'democratic', 'presidential', 'nominee', 'hillary', 'clinton', 'fact', 'information', 'demand', 'send', 'signal', 'team', 'trump', 'notably', 'fire', 'special', 'counsel', 'robert', 'mueller', 'circumstance', 'despite', 'fact', 'trump', 'white', 'house', 'lay', 'groundwork', 'speak', 'tweet', 'grassley', 'warning', 'think', 'senator', 'grassley', 'rest', 'senate', 'need', 'look', 'warning', 'give', 'trump', 'jr', 'manafort', 'follow', 

In [17]:
textos_limpios_str = [" ".join(tokens) for tokens in limpiar_textos]

In [20]:
data_limpio = pd.DataFrame({
    'text_limpio': textos_limpios_str,
    'label': data['label'].values
})
data_limpio.to_csv("../data/dataset_limpio.csv", index=False, encoding='utf-8')
data_limpio.head()

,text_limpio,label
0,donald trump white house chaos try cover russi...,0
1,donald trump presumptive gop nominee time reme...,0
2,mike pence huge homophobe support ex gay conve...,0
3,san francisco reuters california attorney gene...,1
4,twisted reasoning come pelosi day especially p...,0
